# Outlier Analysis
- outlier: 나머지 데이터와 수치적으로 멀리 떨어져 있는 관측치거나 간단히 말해서 범위를 벗어나는 값

## CAUSE FOR OUTLIERS 
- Data Entry Errors: 데이터 수집, 기록 또는 입력 중에 발생한 오류와 같은 entry 오류로 인한 데이터 이상값
- Measurement Error: measurement instrument(계측기)의 결함
- Natural Outlier: outlier가 오차로 인해 인공적이지 않을 때 발생, 대부분의 실제 데이터가 이 범주에 속함

## OUTLIER DETECTION 
- outlier의 두가지 유형: Univariate / Multivariate
- Unvariate: 단일 변수의 분포를 찾아볼 때 사용
- Multivariate: n차원 공간의 outlier

## DIFFERENT OUTLIER DETECTION TECHNIQUE. 
1. Hypothesis Testing
2. Z-score method
3. Robust Z-score
4. I.Q.R method
5. Winsorization method(Percentile Capping)
6. DBSCAN Clustering
7. Isolation Forest
8. Visualizing the data
   
 
## 1. Hypothesis testing (Grubbs test)
$$
\begin{array}{l}{\text { Grubbs' test is defined for the hypothesis: }} \\ {\begin{array}{ll}{\text { Ho: }}  {\text { There are no outliers in the data set }} \\ {\mathrm{H}_{\mathrm{1}} :}  {\text { There is exactly one outlier in the data set }}\end{array}}\end{array}
$$
$$
\begin{array}{l}{\text {The Grubbs' test statistic is defined as: }} \\ {\qquad G_{calculated}=\frac{\max \left|X_{i}-\overline{X}\right|}{SD}} \\ {\text { with } \overline{X} \text { and } SD \text { denoting the sample mean and standard deviation, respectively. }} \end{array}
$$
$$
G_{critical}=\frac{(N-1)}{\sqrt{N}} \sqrt{\frac{\left(t_{\alpha /(2 N), N-2}\right)^{2}}{N-2+\left(t_{\alpha /(2 N), N-2}\right)^{2}}}
$$

\begin{array}{l}{\text { If the calculated value is greater than critical, you can reject the null hypothesis and conclude that one of the values is an outlier }}\end{array}

In [1]:
import numpy as np
import scipy.stats as stats
x = np.array([12,13,14,19,21,23])
y = np.array([12,13,14,19,21,23,45])
def grubbs_test(x):
    n = len(x)
    mean_x = np.mean(x)
    sd_x = np.std(x)
    numerator = max(abs(x-mean_x))
    g_calculated = numerator/sd_x
    print("Grubbs Calculated Value:",g_calculated)
    t_value = stats.t.ppf(1 - 0.05 / (2 * n), n - 2)
    g_critical = ((n - 1) * np.sqrt(np.square(t_value))) / (np.sqrt(n) * np.sqrt(n - 2 + np.square(t_value)))
    print("Grubbs Critical Value:",g_critical)
    if g_critical > g_calculated:
        print("From grubbs_test we observe that calculated value is lesser than critical value, Accept null hypothesis and conclude that there is no outliers\n")
    else:
        print("From grubbs_test we observe that calculated value is greater than critical value, Reject null hypothesis and conclude that there is an outliers\n")
grubbs_test(x)
grubbs_test(y)

Grubbs Calculated Value: 1.4274928542926593
Grubbs Critical Value: 1.887145117792422
From grubbs_test we observe that calculated value is lesser than critical value, Accept null hypothesis and conclude that there is no outliers

Grubbs Calculated Value: 2.2765147221587774
Grubbs Critical Value: 2.019968507680656
From grubbs_test we observe that calculated value is greater than critical value, Reject null hypothesis and conclude that there is an outliers



## 2. Z-score method
<img style="float: center;"  src="https://i.pinimg.com/originals/cd/14/73/cd1473c4c82980c6596ea9f535a7f41c.jpg" width="350px">

- z-score를 사용하면 평균에서 몇 개의 표준 편차를 얻을 수 있는지 알 수 있음
- 위그림은 정규 곡선 아래의 면적과 해당 표준 편차가 적용되는 면적을 나타냅니다.
1. 데이터 점의 68%가 + 또는 -1 표준 편차 사이에 있습니다.
2. 데이터 점의 95%가 + 또는 -2 표준 편차 사이에 있습니다.
3. 데이터 점의 99.7%가 + 또는 -3 표준 편차 사이에 있습니다.
- 데이터 점의 z 점수가 3보다 크면(면적의 99.7%를 차지하므로) 데이터 값이 다른 값과 상당히 다르다는 것을 나타내고 특이치로 간주된다.

### Z-score formula: ${z score=\frac{ X - Mean}{Standard Deviation}}$

In [5]:
import pandas as pd
import numpy as np

data = pd.read_csv('../creditcard.csv')
out = []

def Zscore_outlier(df):
    m = np.mean(df)
    sd = np.std(df)
    for i in df: 
        z = (i-m)/sd
        if np.abs(z) > 3: 
            out.append(i)
    return out

print('Outlier 개수:', len(Zscore_outlier(data['Amount'])))

Outlier 개수: 4076


## 3. Robust z-score
- median absolute deviation method
- 파라미터가 일부 변경되는 Z-score method와 비슷함
- 평균 및 표준 편차는 outlier의 영향을 많이 받으므로 이 모수를 변경해서 중위수 및 중위수에서 절대 편차를 사용

\begin{array}{l} {R.Z.score=\frac{0.6745*( X_{i} - Median)}{MAD}}  \end{array}

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats

data = pd.read_csv('../creditcard.csv')
out = []

def ZRscore_outlier(df):
    med = np.median(df)
    ma = stats.median_abs_deviation(df)
    
    for i in df: 
        z = (0.6745*(i-med))/ (np.median(ma))
        if np.abs(z) > 3: 
            out.append(i)
    return out
    
print('Outlier 개수:', len(ZRscore_outlier(data['Amount'])))

Outlier 개수: 52062


## 4. IQR method
<img style="float: center;"  src=" https://miro.medium.com/max/18000/1*2c21SkzJMf3frPXPAR_gZA.png" width="400px">

- (–1.5×IQR) ~ (+1.5×IQR) 범위 벗어나는 모든 값을 outlier로 처리
- Q1 = 제1 사분위수 = data의 25%
- Q2 = 제2 사분위수 = data의 50% = median
- Q3 = 제3 사분위수 = data의 75%
- (Q1–1.5*IQR): dataset에서 가장 작은 값, (Q3+1.5*IQR): dataset에서 가장 큰 값

In [8]:
import pandas as pd
import numpy as np

data = pd.read_csv('../creditcard.csv')
out = []

def iqr_outliers(df):
    
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    
    iqr = q3-q1
    
    Lower_tail = q1 - 1.5 * iqr
    Upper_tail = q3 + 1.5 * iqr
    
    for i in df:
        if i > Upper_tail or i < Lower_tail:
            out.append(i)
    return out

print('Outlier 개수:', len(iqr_outliers(data['Amount'])))

Outlier 개수: 31904


## 5. Winsorization method (percentile capping)
- IQR method와 비슷함
- (data의 1%) ~ (data의 99%) 범위 벗어나는 모든 값을 outlier로 처리

In [9]:
import pandas as pd
import numpy as np

data = pd.read_csv('../creditcard.csv')
out = []

def Winsorization_outliers(df):
    q1 = np.percentile(df , 1)
    q3 = np.percentile(df , 99)
    for i in df:
        if i > q3 or i < q1:
            out.append(i)
    return out

print('Outlier 개수:', len(Winsorization_outliers(data['Amount'])))

Outlier 개수: 5618


## 6. DBSCAN
<img style="float: center;"  src="https://qphs.fs.quoracdn.net/main-qimg-384458d7ab61f88e443b5e99bcd06622" width="400px">

- DBSCAN은 데이터 세트를 고밀도 영역의 하위 그룹으로 나누고 고밀도 영역 클러스터를 이상값으로 식별하는 밀도 기반 클러스터링 알고리즘
- 여기서 군집 -1은 군집에 특이치가 포함되어 있고 나머지 군집에는 특이치가 없음을 나타냅니다 (kmeans clustering과 유사)
- 다변량 이상값 감지에 대한 최상의 결과를 제공
- DBSCAN에는 두 가지 매개 변수가 필요
1. epsilon: 근처 이웃을 검색할 반경을 정의하는 거리 매개 변수.
2. 클러스터를 구성하는 데 필요한 최소 포인트 양

**엡실론 및 minPts를 사용하여 각 데이터 지점을 다음과 같이 분류**
- Core point: 반경 내에 최소 개수의 다른 포인트(minPts)가 있는 포인트.
- border point: 점은 코어 점의 반경 내에 있지만 자체 반경 내에 다른 최소 포인트 수(minPts)보다 작습니다.
- noise point: 핵심 포인트 또는 경계 포인트가 아닌 포인트

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN

data = pd.read_csv('../creditcard.csv')

def DB_outliers(df):
    outlier_detection = DBSCAN(eps = 2, metric='euclidean', min_samples = 5)
    clusters = outlier_detection.fit_predict(df.values.reshape(-1,1))
    data = pd.DataFrame()
    data['cluster'] = clusters
    return data['cluster'].value_counts().sort_values(ascending=False)
    
DB_outliers(data['Amount']) 

## 7. Isolation forest
<img style="float: center;"  src="https://miro.medium.com/max/875/0*0GuMixLdSZo3V3Nh." width="450px">

- random forest와 유사
1. 데이터 점을 outlier와 not oulier로 분류하고 매우 높은 차원 데이터에서 잘 작동
2. 의사결정 트리를 기반으로 작동하며 이상값을 격리(isolate)
3. 결과가 -1이면 이 특정 데이터 점이 특이치임을 의미합니다. 결과가 1이면 데이터 점이 특이치가 아님을 의미합니다.

In [ ]:
from sklearn.ensemble import IsolationForest
import numpy as np
import pandas as pd

data = pd.read_csv('../creditcard.csv')

train['Fare'].fillna(train[train.Pclass==3]['Fare'].median(),inplace=True)
def Iso_outliers(df):
    iso = IsolationForest( behaviour = 'new', random_state = 1, contamination= 'auto')
    preds = iso.fit_predict(df.values.reshape(-1,1))
    data = pd.DataFrame()
    data['cluster'] = preds
    print(data['cluster'].value_counts().sort_values(ascending=False))
Iso_outliers(train['Fare']) 

## 8. Visualizing the data
- 데이터 시각화는 데이터 정리, 탐색, 이상치 탐지, 클러시터 식별에 유용
1. Box and whisker plot (box plot)
2. Scatter plot.
3. Histogram.
4. Distribution Plot.
5. QQ plot.

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.graphics.gofplots import qqplot

data = pd.read_csv('../creditcard.csv')

def Box_plots(df):
    plt.figure(figsize=(10, 4))
    plt.title("Box Plot")
    sns.boxplot(df)
    plt.show()

def hist_plots(df):
    plt.figure(figsize=(10, 4))
    plt.hist(df)
    plt.title("Histogram Plot")
    plt.show()

def scatter_plots(df1,df2):
    fig, ax = plt.subplots(figsize=(10,4))
    ax.scatter(df1,df2)
    ax.set_xlabel('Age')
    ax.set_ylabel('Fare')
    plt.title("Scatter Plot")
    plt.show()

def dist_plots(df):
    plt.figure(figsize=(10, 4))
    sns.distplot(df)
    plt.title("Distribution plot")
    sns.despine()
    plt.show()

def qq_plots(df):
    plt.figure(figsize=(10, 4))
    qqplot(df,line='s')
    plt.title("Normal QQPlot")
    plt.show()


Box_plots(data['Amount'])
hist_plots(data['Amount'])
scatter_plots(data['Amount'],data['Time'])
dist_plots(data['Amount'])
qq_plots(data['Amount'])

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<Figure size 1000x400 with 1 Axes>

<Figure size 1000x400 with 1 Axes>

<Figure size 1000x400 with 1 Axes>

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<Figure size 1000x400 with 1 Axes>

<Figure size 1000x400 with 0 Axes>

<Figure size 640x480 with 1 Axes>

## 다음 할 일: outlier 지우기
* 특이치는 데이터 세트의 평균 및 표준 편차에 좋지 않습니다. 이는 통계적으로 잘못된 결과를 제공할 수 있다.
* 오차분산을 증가시키고 통계적 검정의 검정력을 감소시킨다.
* 특이치가 랜덤하지 않게 분포되어 있으면 정규성이 저하될 수 있습니다.
* 대부분의 기계 학습 알고리즘은 특이치가 있는 곳에서는 제대로 작동하지 않습니다. 따라서 특이치를 감지하고 제거하는 것이 바람직하다.
* 회귀 분석, 분산 분석 및 기타 통계 모형 가정의 기본 가정에도 영향을 줄 수 있습니다.
이러한 모든 이유로 우리는 통계/기계 학습 모델을 구축하기 전에 특이치를 주의하고 특이치를 처리해야 합니다. 이상값을 처리하는 데 사용되는 몇 가지 기술들이 있다.
1. Deleting observations
2. Transforming values
3. Imputation
4. Separately treating


### 9-1 Deleting observations
데이터 입력 오류, 데이터 처리 오류 또는 특이치 관측치의 수가 매우 적기 때문에 특이치 값을 삭제합니다. 이상값을 제거하기 위해 양쪽 끝을 다듬을 수도 있습니다. 그러나 작은 데이터 집합이 있는 경우 관찰 내용을 삭제하는 것은 좋지 않습니다.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

train = pd.read_csv('../input/cost-of-living/cost-of-living-2018.csv')
sns.boxplot(train['Cost of Living Index'])
plt.title("Box Plot before outlier removing")
plt.show()

def drop_outliers(df, field_name):
    iqr = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
    df.drop(df[df[field_name] > (iqr + np.percentile(df[field_name], 75))].index, inplace=True)
    df.drop(df[df[field_name] < (np.percentile(df[field_name], 25) - iqr)].index, inplace=True)
    
drop_outliers(train, 'Cost of Living Index')
sns.boxplot(train['Cost of Living Index'])
plt.title("Box Plot after outlier removing")
plt.show()

## Transforming values

변수를 변환하면 특이치도 제거할 수 있다. 이러한 변환된 값은 극단값으로 인한 변동을 줄인다.
1. Scaling
2. Log transformation
3. Cube Root normalization
4. Box-Cox transformation

- 이러한 기술은 데이터 집합의 값을 더 작은 값으로 변환합니다.
- 데이터가 극단값을 너무 많이 갖거나 치우친 경우 이 방법을 사용하면 데이터가 정규 분포를 따르도록 할 수 있습니다.
- 그러나 이러한 기술이 항상 최상의 결과를 제공하는 것은 아닙니다.
- 이 방법에서는 데이터가 손실되지 않습니다.
- 이 모든 방법에서 cox 변환은 최상의 결과를 제공합니다.

In [ ]:
#Scalling
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing

train = pd.read_csv('../input/cost-of-living/cost-of-living-2018.csv')

plt.hist(train['Cost of Living Index'])
plt.title("Histogram before Scalling")
plt.show()
scaler = preprocessing.StandardScaler()
train['Cost of Living Index'] = scaler.fit_transform(train['Cost of Living Index'].values.reshape(-1,1))
plt.hist(train['Cost of Living Index'])
plt.title("Histogram after Scalling")
plt.show()

In [ ]:
#Log Transformation
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

train = pd.read_csv('../input/cost-of-living/cost-of-living-2018.csv')
sns.distplot(train['Cost of Living Index'])
plt.title("Distribution plot before Log transformation")
sns.despine()
plt.show()
train['Cost of Living Index'] = np.log(train['Cost of Living Index'])
sns.distplot(train['Cost of Living Index'])
plt.title("Distribution plot after Log transformation")
sns.despine()
plt.show()

In [ ]:
#cube root Transformation
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

train = pd.read_csv('../input/titanic/train.csv')
plt.hist(train['Age'])
plt.title("Histogram before cube root Transformation")
plt.show()
train['Age'] = (train['Age']**(1/3))
plt.hist(train['Age'])
plt.title("Histogram after cube root Transformation")
plt.show()

In [ ]:
#Box-transformation
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy

train = pd.read_csv('../input/cost-of-living/cost-of-living-2018.csv')
sns.boxplot(train['Rent Index'])
plt.title("Box Plot before outlier removing")
plt.show()
train['Rent Index'],fitted_lambda= scipy.stats.boxcox(train['Rent Index'] ,lmbda=None)
sns.boxplot(train['Rent Index'])
plt.title("Box Plot after outlier removing")
plt.show()

## Imputation

결측값의 imputation처럼 특이치도 imputation시킬 수 있습니다. 이 방법에서는 평균, 중위수, 0 값을 사용할 수 있습니다. 데이터 손실은 없으며 여기서 중위수는 특이치의 영향을 받지 않으므로 적합하다.

In [ ]:
#mean imputation
import pandas as pd
import numpy as np
train = pd.read_csv('../input/titanic/train.csv')
sns.boxplot(train['Age'])
plt.title("Box Plot before mean imputation")
plt.show()
q1 = train['Age'].quantile(0.25)
q3 = train['Age'].quantile(0.75)
iqr = q3-q1
Lower_tail = q1 - 1.5 * iqr
Upper_tail = q3 + 1.5 * iqr
m = np.mean(train['Age'])
for i in train['Age']:
    if i > Upper_tail or i < Lower_tail:
            train['Age'] = train['Age'].replace(i, m)
sns.boxplot(train['Age'])
plt.title("Box Plot after mean imputation")
plt.show()   

In [ ]:
#median imputation
import pandas as pd
import numpy as np
train = pd.read_csv('../input/titanic/train.csv')
sns.boxplot(train['Age'])
plt.title("Box Plot before median imputation")
plt.show()
q1 = train['Age'].quantile(0.25)
q3 = train['Age'].quantile(0.75)
iqr = q3-q1
Lower_tail = q1 - 1.5 * iqr
Upper_tail = q3 + 1.5 * iqr
med = np.median(train['Age'])
for i in train['Age']:
    if i > Upper_tail or i < Lower_tail:
            train['Age'] = train['Age'].replace(i, med)
sns.boxplot(train['Age'])
plt.title("Box Plot after median imputation")
plt.show()            


In [ ]:
#Zero value imputation
import pandas as pd
import numpy as np
train = pd.read_csv('../input/titanic/train.csv')
sns.boxplot(train['Age'])
plt.title("Box Plot before Zero value imputation")
plt.show()
q1 = train['Age'].quantile(0.25)
q3 = train['Age'].quantile(0.75)
iqr = q3-q1
Lower_tail = q1 - 1.5 * iqr
Upper_tail = q3 + 1.5 * iqr
for i in train['Age']:
    if i > Upper_tail or i < Lower_tail:
            train['Age'] = train['Age'].replace(i, 0)
sns.boxplot(train['Age'])
plt.title("Box Plot after Zero value imputation")
plt.show()            


### seperately treating
이상치의 수가 유의하고 데이터 집합이 작으면 통계 모형에서 특이치를 별도로 처리해야 합니다.
접근법 중 하나는 두 그룹을 서로 다른 두 그룹으로 취급하고 두 그룹에 대한 개별 모델을 구축한 다음 출력을 결합하는 것입니다.
그러나 데이터 세트가 클 경우 이 기술은 지루합니다

# Conclusion
1. median은 데이터에 outlier가 있거나 skewed된 경우 중심 경향을 측정하는 가장 좋은 척도
2. Winsorization Method 또는 Percentile Caping은 이상값 검출 기법보다 우수합니다.
3. 중위 귀속은 특이치를 완전히 제거한다.
특이치는 기계 학습의 주요 문제 중 하나이다. 특이치 결과를 무시하면 모형의 성능이 저하됩니다. 이 커널에서 특이치, 특이치 탐지, 특이치 처리 기법과 관련된 거의 모든 항목을 다루려고 합니다.

## Reference
- https://medium.com/datadriveninvestor/finding-outliers-in-dataset-using-python-efc3fce6ce32
- http://www.askanalytics.in/p/outlier-treatment.html
- https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
- https://www.kdnuggets.com/2018/12/four-techniques-outlier-detection.html